# Creating a Custom Dense Layer

In this notebook, we will advance from using simple Lambda layers to developing a custom layer that inherits from the Layer class in Keras. This type of layer allows for more sophisticated functionality, including the incorporation of trainable weights that adjust during the model's training process. Unlike the stateless operations implemented with Lambda layers, these custom layers can maintain and update state across the training lifecycle. This feature is crucial for creating complex neural network components that need to learn from the data iteratively. Let’s dive into the details of how to construct such a layer and integrate it into our model.

## Imports

In [ ]:
import tensorflow as tf
import numpy as np

## Custom Layer with weights

To create a trainable custom layer in Keras, we start by defining a class that inherits from Keras's Layer base class. This structure allows us to construct layers with customizable behaviors and properties. In the class declaration, there are three essential methods we need to implement: __init__(), build(), and call(). These functions are critical for establishing both the structure and functionality of the layer:

- __init__(): This method initializes the layer, setting up any parameters or sub-layers that are part of its architecture.
- build(self, input_shape): Called once from the first call to call(), it is where we define the weights of the layer—these weights are trainable and are what allow the layer to learn from the input data.
- call(self, inputs): This method contains the computation logic of the layer. It is run every time the layer is called and is where the layer's logic interacts with the input tensor to produce an output tensor.

By defining these methods, our custom layer will not only perform specific computations but also adapt through training, thanks to its trainable weights. Let's explore how these components come together to enhance the capabilities of our neural network.

In [ ]:
# Inherit from this base class
from tensorflow.keras.layers import Layer

class SimpleDense(Layer):

    def __init__(self, units=32):
        '''Initializes the instance attributes'''
        super(SimpleDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        '''Create the state of the layer (weights)'''
        # Initialize the weights
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(name="kernel",
            initial_value=w_init(shape=(input_shape[-1], self.units),
                                 dtype='float32'),
            trainable=True)

        # Initialize the biases
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(name="bias",
            initial_value=b_init(shape=(self.units,), dtype='float32'),
            trainable=True)

    def call(self, inputs):
        '''Defines the computation from inputs to outputs'''
        return tf.matmul(inputs, self.w) + self.b

Now that we've defined our custom layer, integrating it into a neural network model is straightforward. We can use the custom layer just like any standard layer provided by Keras. Below, I'll demonstrate how to incorporate our newly created custom layer into a model's architecture. Here's an example:

In [ ]:
# Declare an instance of the class
my_dense = SimpleDense(units=1)

# Define an input and feed into the layer
x = tf.ones((1, 1))
y = my_dense(x)

# Print parameters of the base Layer class like `variables` can be used
print(my_dense.variables)

[<tf.Variable 'simple_dense/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[-0.05299155]], dtype=float32)>, <tf.Variable 'simple_dense/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]


Now, let's apply our custom layer within a simple network to see it in action. This step will help us understand how the custom layer behaves when integrated with other standard layers in a practical setting. We'll construct a straightforward neural network that includes our custom layer to assess its performance and functionality. Here's how we can set up the network:

In [ ]:
# Define the dataset
xs = np.array([[-1.0],  [0.0], [1.0], [2.0], [3.0], [4.0]], dtype=float)
ys = np.array([-3.0, -1.0, 1.0, 3.0, 5.0, 7.0], dtype=float)


# Use the Sequential API to build a model with our custom layer
my_layer = SimpleDense(units=1)
model = tf.keras.Sequential([my_layer])

# Configure and train the model
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(xs, ys, epochs=500,verbose=0)

# Perform inference
print(model.predict([10.0]))

# See the updated state of the variables
print(my_layer.variables)

1/1 [==============================] - 0s 141ms/step
[[18.981592]]
[<tf.Variable 'simple_dense_1/kernel:0' shape=(1, 1) dtype=float32, numpy=array([[1.9973322]], dtype=float32)>, <tf.Variable 'simple_dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([-0.9917294], dtype=float32)>]
